In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt

# 1. Đọc và chuẩn hóa dữ liệu
data = pd.read_csv('../../data/ai4i2020.csv')
features = data.drop(columns=['UDI', 'Product ID', 'Type', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'])
labels = data['Machine failure']

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# 2. Chia dữ liệu: Train với 8000 dòng bình thường và Test với 200 bình thường + 200 bất thường
X_train = scaled_features[labels == 0][:8000]
normal_test = scaled_features[labels == 0][8000:8200]
anomaly_test = scaled_features[labels == 1][:200]

# Gộp tập kiểm tra (400 dòng)
X_test = np.vstack([normal_test, anomaly_test])
y_test = np.hstack([np.zeros(200), np.ones(200)])  # Nhãn tương ứng

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# 1. Định nghĩa Generator
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# 2. Định nghĩa Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# 3. Chuẩn bị dữ liệu
# Giả sử X_train_tensor và X_test_tensor đã được định nghĩa trước đó
# X_train_tensor chứa 200 dòng bình thường và 200 dòng bất thường

# 4. Khởi tạo mô hình và optimizer
latent_dim = 10
input_dim = X_train_tensor.shape[1]

generator = Generator(latent_dim, input_dim)
discriminator = Discriminator(input_dim)

optimizer_g = optim.Adam(generator.parameters(), lr=0.001)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.001)

# 5. Huấn luyện GAN với 200 dòng bình thường và 200 dòng bất thường
epochs = 50
batch_size = 32
train_loader = torch.utils.data.DataLoader(X_train_tensor, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    for real_data in train_loader:
        # Huấn luyện Discriminator
        optimizer_d.zero_grad()
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        noise = torch.randn(batch_size, latent_dim)
        fake_data = generator(noise)

        real_loss = nn.BCELoss()(discriminator(real_data), real_labels)
        fake_loss = nn.BCELoss()(discriminator(fake_data.detach()), fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_d.step()

        # Huấn luyện Generator
        optimizer_g.zero_grad()
        noise = torch.randn(batch_size, latent_dim)
        generated_data = generator(noise)
        g_loss = nn.BCELoss()(discriminator(generated_data), real_labels)
        g_loss.backward()
        optimizer_g.step()

    print(f'Epoch [{epoch + 1}/{epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')



Epoch [1/50], d_loss: 0.4495, g_loss: 2.0996
Epoch [2/50], d_loss: 1.2542, g_loss: 0.7439
Epoch [3/50], d_loss: 1.1118, g_loss: 1.0497
Epoch [4/50], d_loss: 1.6407, g_loss: 0.6447
Epoch [5/50], d_loss: 1.7562, g_loss: 0.7462
Epoch [6/50], d_loss: 1.1265, g_loss: 0.8915
Epoch [7/50], d_loss: 0.8485, g_loss: 1.1758
Epoch [8/50], d_loss: 0.7988, g_loss: 1.1686
Epoch [9/50], d_loss: 1.0042, g_loss: 1.0769
Epoch [10/50], d_loss: 1.2949, g_loss: 0.8027
Epoch [11/50], d_loss: 1.1504, g_loss: 0.8411
Epoch [12/50], d_loss: 1.3108, g_loss: 0.8486
Epoch [13/50], d_loss: 1.1964, g_loss: 0.7963
Epoch [14/50], d_loss: 1.0713, g_loss: 0.8315
Epoch [15/50], d_loss: 1.1918, g_loss: 0.8528
Epoch [16/50], d_loss: 1.0980, g_loss: 0.9195
Epoch [17/50], d_loss: 1.1398, g_loss: 0.9509
Epoch [18/50], d_loss: 1.1894, g_loss: 0.8697
Epoch [19/50], d_loss: 1.0508, g_loss: 0.9296
Epoch [20/50], d_loss: 1.0233, g_loss: 1.0163
Epoch [21/50], d_loss: 0.9673, g_loss: 1.0375
Epoch [22/50], d_loss: 1.1540, g_loss: 0.94

In [16]:
# 6. Dự đoán trên tập kiểm tra
discriminator.eval()
with torch.no_grad():
    test_probs = discriminator(X_test_tensor).numpy().flatten()

# 7. Đánh dấu dữ liệu dựa trên xác suất P
threshold = 0.5  # Nếu P >= 0.5 thì bình thường, ngược lại bất thường
y_pred = (test_probs >= threshold).astype(int)

# 8. In Confusion Matrix và Classification Report
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 9. Tính ROC AUC Score
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC Score: {roc_auc}")


ValueError: Found input variables with inconsistent numbers of samples: [400, 339]

In [ ]:

# 10. Vẽ biểu đồ phân phối xác suất dự đoán
plt.figure(figsize=(10, 6))
plt.hist(test_probs, bins=50, alpha=0.75, label='Discriminator Probabilities')
plt.axvline(threshold, color='red', linestyle='dashed', linewidth=2, label=f'Threshold ({threshold})')
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.title('Distribution of Discriminator Probabilities')
plt.legend()
plt.show()